In [179]:
import numpy as np
import pandas as pd
import datetime
import math

In [207]:
dam = pd.read_csv('dam_out.csv')
dam

,year,month,day,date,market,hour,price,volume,day_of_week,value,price_min,price_max,price_mean,price_morning_max,price_evening_max,cat
0,2016,1,1,2016-01-01,1,0,40.10,477.5,4,19147.750,35.09,60.08,45.98250,48.03,60.08,off
1,2016,1,1,2016-01-01,1,1,40.08,486.6,4,19502.928,35.09,60.08,45.98250,48.03,60.08,off
2,2016,1,1,2016-01-01,1,2,40.06,495.6,4,19853.736,35.09,60.08,45.98250,48.03,60.08,off
3,2016,1,1,2016-01-01,1,3,40.08,496.6,4,19903.728,35.09,60.08,45.98250,48.03,60.08,off
4,2016,1,1,2016-01-01,1,4,40.07,502.9,4,20151.203,35.09,60.08,45.98250,48.03,60.08,off
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46723,2021,4,29,2021-04-29,1,4,39.97,126.1,3,5040.217,39.79,296.82,98.74250,195.01,296.82,off
46724,2021,4,29,2021-04-29,1,12,68.06,71.0,3,4832.260,39.79,296.82,98.74250,195.01,296.82,standard
46725,2021,4,30,2021-04-30,1,0,44.04,59.6,4,2624.784,38.97,295.98,98.21875,190.00,295.98,off
46726,2021,4,30,2021-04-30,1,8,190.00,72.0,4,13680.000,38.97,295.98,98.21875,190.00,295.98,morning


In [213]:
price = pd.read_csv('dam.csv')
price.rename(columns={'tradedate': 'date'}, inplace=True)
price=price.set_index([pd.DatetimeIndex(price['date']),'hour'])
price

hourlyid  market        date   price  volume
date       hour                                              
2016-01-01 0        26345       1  2016-01-01   40.10   477.5
           1        26346       1  2016-01-01   40.08   486.6
           2        26347       1  2016-01-01   40.06   495.6
           3        26348       1  2016-01-01   40.08   496.6
           4        26349       1  2016-01-01   40.07   502.9
...                   ...     ...         ...     ...     ...
2021-04-29 4        73029       1  2021-04-29   39.97   126.1
           12       73037       1  2021-04-29   68.06    71.0
2021-04-30 0        73049       1  2021-04-30   44.04    59.6
           8        73057       1  2021-04-30  190.00    72.0
           17       73066       1  2021-04-30   67.97   114.2

[46728 rows x 5 columns]

In [214]:
flow = pd.read_csv('flow.csv')

flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=price['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.reset_index()
flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-02,13,50,BOT,ZIMA,2016,1,2,5,50.05,2502.50
1,2016-01-02,14,63,ZIMA,ZIM,2016,1,2,5,50.31,3169.53
2,2016-01-02,15,263,ZIM,ZAMZ,2016,1,2,5,50.49,13278.87
3,2016-01-02,16,100,MOZN_EDM,ZIMA,2016,1,2,5,56.07,5607.00
4,2016-01-02,17,120,ZIM,ZAMZ,2016,1,2,5,56.06,6727.20
...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,3,RSAN,RSAS,2021,3,31,2,148.01,444.03
333321,2021-03-31,20,13,RSAN,BOT,2021,3,31,2,139.06,1807.78
333322,2021-03-31,21,33,MOZS,RSAN,2021,3,31,2,89.03,2937.99
333323,2021-03-31,22,43,ZIMA,ZIM,2021,3,31,2,36.98,1590.14


In [216]:
f_to=flow[['date','hour','flow','value','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,node,year,month,day,day_of_week
0,2016-01-02,13,50,2502.50,ZIMA,2016,1,2,5
1,2016-01-02,14,63,3169.53,ZIM,2016,1,2,5
2,2016-01-02,15,263,13278.87,ZAMZ,2016,1,2,5
3,2016-01-02,16,100,5607.00,ZIMA,2016,1,2,5
4,2016-01-02,17,120,6727.20,ZAMZ,2016,1,2,5
...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,-3,-444.03,RSAN,2021,3,31,2
333321,2021-03-31,20,-13,-1807.78,RSAN,2021,3,31,2
333322,2021-03-31,21,-33,-2937.99,MOZS,2021,3,31,2
333323,2021-03-31,22,-43,-1590.14,ZIMA,2021,3,31,2


In [217]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [219]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,index,flow,value
0,2016-01-01,0,ZAMC,2016,1,1,4,5607,160,6416.00
1,2016-01-01,0,ZAMZ,2016,1,1,4,11320,10,401.00
2,2016-01-01,0,ZIM,2016,1,1,4,5713,-170,-6817.00
3,2016-01-01,1,ZAMC,2016,1,1,4,5714,160,6412.80
4,2016-01-01,1,ZAMZ,2016,1,1,4,11322,10,400.80
...,...,...,...,...,...,...,...,...,...,...
289677,2021-03-31,22,ZAMC,2021,3,31,2,333292,43,1590.14
289680,2021-03-31,23,BOT,2021,3,31,2,666156,81,2993.76
289681,2021-03-31,23,MOZ,2021,3,31,2,666292,-164,-6061.44
289682,2021-03-31,23,NAM,2021,3,31,2,333309,42,1552.32


In [220]:
flows=flows.set_index(['year','month','day','hour'])
dam=dam.set_index(['year','month','day','hour'])

In [221]:
flows

date  node  day_of_week   index  flow    value
year month day hour                                                     
2016 1     1   0    2016-01-01  ZAMC            4    5607   160  6416.00
               0    2016-01-01  ZAMZ            4   11320    10   401.00
               0    2016-01-01   ZIM            4    5713  -170 -6817.00
               1    2016-01-01  ZAMC            4    5714   160  6412.80
               1    2016-01-01  ZAMZ            4   11322    10   400.80
...                        ...   ...          ...     ...   ...      ...
2021 3     31  22   2021-03-31  ZAMC            2  333292    43  1590.14
               23   2021-03-31   BOT            2  666156    81  2993.76
               23   2021-03-31   MOZ            2  666292  -164 -6061.44
               23   2021-03-31   NAM            2  333309    42  1552.32
               23   2021-03-31  ZAMC            2  333231    41  1515.36

[188386 rows x 6 columns]

In [222]:
flows['cat']=dam['cat']
flows.reset_index()

,year,month,day,hour,date,node,day_of_week,index,flow,value,cat
0,2016,1,1,0,2016-01-01,ZAMC,4,5607,160,6416.00,off
1,2016,1,1,0,2016-01-01,ZAMZ,4,11320,10,401.00,off
2,2016,1,1,0,2016-01-01,ZIM,4,5713,-170,-6817.00,off
3,2016,1,1,1,2016-01-01,ZAMC,4,5714,160,6412.80,off
4,2016,1,1,1,2016-01-01,ZAMZ,4,11322,10,400.80,off
...,...,...,...,...,...,...,...,...,...,...,...
188381,2021,3,31,22,2021-03-31,ZAMC,2,333292,43,1590.14,off
188382,2021,3,31,23,2021-03-31,BOT,2,666156,81,2993.76,off
188383,2021,3,31,23,2021-03-31,MOZ,2,666292,-164,-6061.44,off
188384,2021,3,31,23,2021-03-31,NAM,2,333309,42,1552.32,off


In [227]:
flow_monthly=flows.groupby(['year','month','hour', 'node']).sum().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly=flow_monthly.reset_index().drop(['day_of_week','index'],axis=1)
flow_monthly

,node,year,month,hour,flow_sum,value
0,BOT,2016,1,0,-91,-4357.17
1,MOZ,2016,1,0,-1829,-90774.59
2,NAM,2016,1,0,50,2409.80
3,RSA,2016,1,0,-191,-9184.91
4,ZAMC,2016,1,0,1965,92162.31
...,...,...,...,...,...,...
10108,NAM,2021,3,23,1988,86941.09
10109,RSA,2021,3,23,-214,-10236.44
10110,ZAMC,2021,3,23,1955,85881.90
10111,ZAMZ,2021,3,23,-366,-15998.37


In [230]:
flow_monthly.to_csv('dam_flow_monthly.csv')

In [231]:
flow_monthly_cat=flows.groupby(['year','month', 'node','cat']).sum().reset_index(['hour','month','year'])
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat=flow_monthly_cat.reset_index().drop(['day_of_week','index'],axis=1)
flow_monthly_cat

,node,cat,year,month,hour,flow_sum,value
0,BOT,off,2016,1,0,-91,-4357.17
1,MOZ,off,2016,1,0,-1829,-90774.59
2,NAM,off,2016,1,0,50,2409.80
3,RSA,off,2016,1,0,-191,-9184.91
4,ZAMC,off,2016,1,0,1965,92162.31
...,...,...,...,...,...,...,...
17925,ZAMC,off,2021,3,23,1587,69787.47
17926,ZAMC,standard,2021,3,23,368,16094.43
17927,ZAMZ,off,2021,3,23,-180,-7894.95
17928,ZAMZ,standard,2021,3,23,-186,-8103.42


In [232]:
flow_monthly_cat.to_csv('dam_flow_monthly_cat.csv')

In [ ]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]